# Speed of computation

Python, being an interpreted language, tends to be slower than compiled languages like C or Fortran.  Some other languages like Java and Julia tend to use Just-in-Time compilation which can give speedups, but Python also has the problem of being dynamically typed, which eliminates the possibility of many optimizations.

The `timeit` library provides functions to estimate the time taken to run a piece of code.  It can automatically run the code multiple times to get better average results, and can be used to identify bottlenecks in your program.  However, it should be used with care as it is not a detailed function-call-level profiler.

It can either be `import`ed as a module where you can then explicitly called `timeit.timeit(func)` to estimate time for a function, or you can use the *magic syntax* in Python notebooks as shown below.  

In [3]:
import numpy as np
x = np.random.rand(10000,1)

In [4]:
def sumarr(x):
    sum = 0
    for i in range(len(x)): 
    # for i in x:
        sum += x[i]
    return sum
print(sumarr(x))
%timeit sumarr(x)

[5025.43974321]
9.85 ms ± 77.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
import numpy as np
def npsumarr(x):
    return np.sum(x)
print(npsumarr(x))
%timeit npsumarr(x)

5025.439743211509
6.97 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


# Solving equations by Gaussian elimination

Once you have constructed two matrices A and B to represent the system of linear equations 
$$ Ax = b $$
you can then proceed to solve the equations using the process known as Gaussian elimination.

It is assumed you already know how the process works, but to refresh your memory, you could use the reference material at [LibreTexts](https://math.libretexts.org/Bookshelves/Algebra/Book%3A_Algebra_and_Trigonometry_(OpenStax)/11%3A_Systems_of_Equations_and_Inequalities/11.06%3A_Solving_Systems_with_Gaussian_Elimination).

Basically it involves making the A matrix *triangular* and ultimately into the shape of an identity matrix.

In [2]:
# Input matrices - the set of equations - 2 variables x1 and x2
A = [ [2,3], [1,-1] ]
B = [6,1/2]
print(A)
print(B)

[[2, 3], [1, -1]]
[6, 0.5]


In [7]:
# Normalize row 1
norm = A[0][0]
for i in range(len(A[0])): A[0][i] /= norm
B[0] = B[0]/norm
print(A)
print(B)

[[1.0, 1.5], [1, -1]]
[3.0, 0.5]


In [8]:
# Eliminate row 2 - A[1] - need to check and ensure div-by-zero etc doesnt happen
norm = A[1][0] / A[0][0]
for i in range(len(A[1])): A[1][i] = A[1][i] - A[0][i] * norm
B[1] = B[1] - B[0] * norm
print(A)
print(B)

[[1.0, 1.5], [0.0, -2.5]]
[3.0, -2.5]


In [9]:
# Normalize row 2 - B[1] will now contain the solution for x2
norm = A[1][1]
for i in range(len(A[1])): A[1][i] = A[1][i] / norm
B[1] = B[1] / norm
print(A)
print(B)

[[1.0, 1.5], [-0.0, 1.0]]
[3.0, 1.0]


In [10]:
# Sub back and solve for B[0] <-> x1
# This can be seen as eliminating A[0][1]
norm = A[0][1] / A[0][0]
# note that len(A) will give number of rows
for i in range(len(A)): 
    A[i][1] = A[i][1] - A[i][0] * norm
    B[i] = B[i] - A[i][0] * norm
print(A)
print(B)

[[1.0, 0.0], [-0.0, 1.0]]
[1.5, 1.0]


## Problems with Gaussian elimination

There are several obvious problems with the method outlined here.  These include:

- Performance - Python lists are not the most efficient way to store matrices
- Zeros: the simple example does not consider a scenario where one of the values on the diagonal may be 0.  Then some shuffling of rows is required.
- Numerical stability: there are several *normalization* steps involved, where it is quite possible for the values to blow up out of control if not managed properly.  Usually some kind of pivoting techniques are used to get around these issues.

In [11]:
import numpy as np
A1 = np.array( [ [2,3], [1,-1] ] )
B1 = np.array( [6, 1/2] )
np.linalg.solve(A1, B1)

array([1.5, 1. ])

# Library functions

*Numeric Python* or `numpy` is a library that allows Python code to directly call highly efficient implementations of several linear algebra routines (that have been written and optimized using C or Fortran and generally offer very high performance).

Although you can use the same `list` based approach to create matrices, it is better to declare them as the `array` data type:

- the numeric `type` (float, int etc.) can be specified for arrays
- memory allocation is done more efficiently by assuming they are not meant to be arbitrarily extended or changed

# SPICE basics

Our goal is to implement a SPICE simulator.  In order to do this, we first need to read in the circuit description from a text file.  To start with, we will only consider the basic elements of SPICE: Voltage sources, Current sources, and Resistors.  A typical SPICE netlist would look like this:

```spice
.circuit
R1 GND 1 1  
R2 1 2 1    
V1 GND 2 dc 2
.end
```

This is basically a *netlist* with 3 *nodes* - one of which is Ground (GND) which is assumed to be have a voltage of 0V.  We can write down Kirchhoff's current law (KCL) equations at each node, to account for current balance.  In addition, we will have some equations that specify the voltages between nodes having a direct voltage source, since there is no resistance there to provide an equation.

For the above example, the equations will be

$$
\begin{aligned}
\frac{V1-0}{R1} & + & \frac{V1-V2}{R2} & = & 0 \\
\frac{V2-V1}{R2} & + & I1 & = & 0 \\
V2 & - & 0 & = & 2
\end{aligned}
$$

which can be written in Matrix form as:

$$
\begin{bmatrix}
\frac{1}{R1}+\frac{1}{R2} & \frac{-1}{R2} & 0 \\
\frac{-1}{R2}   & \frac{1}{R2}  & 1 \\
0  & 1  & 0
\end{bmatrix}
\begin{bmatrix}
V1 \\
V2 \\
I1
\end{bmatrix}
=
\begin{bmatrix}
0 \\
0 \\
2
\end{bmatrix}
$$

At this point, you have reduced the solving of the SPICE equations to a known problem (linear equation solving) that you already know how to do.

## AC sources

The assumption above is that the system consists only of Voltage or Current sources and resistors.  What about capacitors, inductors, and AC sources?  These can be handled in exactly the same way as long as the circuit is operating at a single frequency.  We then replace the elements with their corresponding *impedance* values, which are frequency dependent complex numbers, but since there is only a single frequency they will still be constants.

## Problem scenarios

- Voltage source loops
- Current sources at a node
- Circuit defined with both DC and AC sources
- Syntax errors

# String and File manipulation

Given a SPICE netlist like the one above, you need to *read* it and construct an internal matrix as described above.  For string manipulation, there are a few helpful utility functions that we can see here.

In [12]:
circ = """.circuit
R1 GND 1 1  
R2 1 2 1    
V1 GND 2 dc 2
.end
""".splitlines()
for l in circ:
    if l[0] == 'R':
        print("Found a resistor")
    elif l[0] == 'V':
        print("Found a voltage source with value: ", float(l.split()[4]))

Found a resistor
Found a resistor
Found a voltage source with value:  2.0


## Files

You can read from a file using the `readlines()` method of file objects.  One thing to keep in mind is how you open and close file objects.  In particular, it is strongly recommended to use the pattern `with open("filename") as f:` to ensure that the file is closed once you are done with reading it.  

# Assignment

The following are the problems you need to solve for this assignment.  You need to submit your code (either as standalone Python script or a Python notebook), a PDF document explaining your solution (either generated from the notebook or a separate LaTeX document), and any supporting files you may have (such as circuit netlists you used for testing your code).

- Write a function to find the factorial of N (N being an input) and find the time taken to compute it.  This will obviously depend on where you run the code and which approach you use to implement the factorial.  Explain your observations briefly.
- Write a linear equation solver that will take in matrices $A$ and $b$ as inputs, and return the vector $x$ that solves the equation $Ax=b$.  Your function should catch errors in the inputs and return suitable error messages for different possible problems.
  - Time your solver to solve a random $10\times 10$ system of equations.  Compare the time taken against the `numpy.linalg.solve` function for the same inputs.
- Given a circuit netlist in the form described above, read it in from a file, construct the appropriate matrices, and use the solver you have written above to obtain the voltages and currents in the circuit.  If you find AC circuits hard to handle, first do this for pure DC circuits, but you should be able to handle both voltage and current sources.

## Bonus assignments

- (Small bonus): after reading in the netlist, allow some or all sources and impedances to be controlled interactively - either using widgets or other mechanisms.  On each change you should recompute the currents and voltages and display them.
- (Large bonus): make a solver that can do real-time transient simulations of a SPICE netlist and update the currents and voltages dynamically.  They should also be plotted as a function of time and react to changes.  This is something along the lines of https://www.falstad.com/circuit/.  Ideally you should be able to do a real-time demo of some experiments you might conduct as part of a basic electronics lab, and simulate the behaviour of an oscilloscope and signal generator.

# Solutions
## Problem 1
In this problem we have to define a function that computes the factorial of a number N and then find the time taken to compute it. Here I am going to use two approaches to find the factorial, one using a straightforward logic and other using recursion. Then I will compare the time taken for output for a common output using timeit function.

In [20]:
def fact_1(N):
    fact=1
    for i in range(N):
        fact *= (i+1)
    return fact
x=50
%timeit fact_1(x)

2.76 µs ± 30.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [21]:
def fact_2(N):
    if(N==0):
        return 1
    else:
        return N*fact_2(N-1)
    pass
x=50
%timeit fact_2(x)

5.29 µs ± 266 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Looking at the results we can infer that for smaller values, both the approaches take equal time but as the values increase, recursion approach takes more time. This is because we to compute the factorial of N we first need to compute the factorial of N-1 and so on.

## Problem 2
Here we have to create a linear equation solver that takes input as two matrices A and B and gives the output matrix x such that Ax=B. 

In [23]:
def eq_solve(coeff_matrix,constant_matrix):
    coeff_matrix=np.array(coeff_matrix,dtype=complex)
    constant_matrix=np.array(constant_matrix,dtype=complex)
    a = "Invalid input!"
    b = "Inconsistent solutions"
    h = len(coeff_matrix)
    l = len(coeff_matrix[0])
    hm = len(constant_matrix)
    if(h!=l | h!=hm):   #making sure that we have a square matrix and that A and B have equal dimensions
        return a
    for i in range(h):
        if(coeff_matrix[i][i]==0):   #checking for 0 diagonal elements
            for i1 in range(i,h):
                if(coeff_matrix[i1][i]!=0):
                    for j1 in range(h):
                        swap = coeff_matrix[i1][j1]
                        coeff_matrix[i1][j1]=coeff_matrix[i][j1]
                        coeff_matrix[i][j1]=swap
                else:
                    return b
        norm = coeff_matrix[i][i]
        for j in range(h):
            coeff_matrix[i][j] /= norm
        constant_matrix[i] /= norm
        for t in range(i+1,h):
            temp=coeff_matrix[t][i]
            for k in range(h):
                coeff_matrix[t][k] -= coeff_matrix[i][k]*temp
            constant_matrix[t] -= temp*constant_matrix[i]
    #At this stage A is an upper triangular matrix
    for i2 in reversed(range(1,h)):  #Converting A to identity matrix
        for t2 in range(i2):
            temp = coeff_matrix[t2][i2]
            coeff_matrix[t2][i2] -= temp * (coeff_matrix[i2][i2])
            constant_matrix[t2] -=  constant_matrix[i2]*temp   
    print(constant_matrix)
    pass
a = np.random.rand(10,10)
b = np.random.rand(10,1)
c = [list(row) for row in a]
d = [row for row in b]
eq_solve(a,b)
np.linalg.solve(c,d)

[[ 3.06633713+0.j]
 [ 0.74596144+0.j]
 [ 0.89528278+0.j]
 [-3.91232152+0.j]
 [ 0.33769881+0.j]
 [ 0.57699274+0.j]
 [-3.03711073+0.j]
 [ 3.13460472+0.j]
 [-1.50890326+0.j]
 [ 0.40563748-0.j]]


array([[ 3.06633713],
       [ 0.74596144],
       [ 0.89528278],
       [-3.91232152],
       [ 0.33769881],
       [ 0.57699274],
       [-3.03711073],
       [ 3.13460472],
       [-1.50890326],
       [ 0.40563748]])

Here eq_solve() is a fuction that successfully solves a system of linear equations using Gaussian elimination. Here we take 2 inputs A and B, where A is the coefficient matrix and B contains the constant terms.
<br>
First we make sure that the matrix A is square matrix and that A and B are of equal dimensions. Then we go through the diagonals one by one. first we scale down the diagonal element to 1 and then we use suitable row operations to make all the elements below the diagonal element equal to 0. This way we convert the matrix to an upper triangular one. If in this process we encounter a diagonal element that is zero, we look at the elements below it and when we encounter one that is zero, we swap the rows. If there exists no such element, the system of equations are inconsistent.
<br>
After this we convert the upper triangular matrix to an identity matrix using row operations. Here, the solution of the system of equations gets stored in the matrix B
<br>
> *Here I have converted the matrices to complex form because it will be useful while solving .SPICE netlists*



## Problem 3
I have successfully implemented a linear equation solver in the previous question. Now using the function we need to solve a circuit using its SPICE directives and give the votages and currents.  

In [32]:
import math
def circuit_solve():
    ckt = input("Enter file containing circuit:") #Taking input for the netlist file
    direct = open(ckt)  #opening the file
    relevant = []
    b= "Invalid input! ( Circuit contains multiple frquencies)"
    dc=0
    ac=0
    flag = False
    for line in direct:  #extracting the circuit info between .circuit and .end
        line = line.strip()
        if(line.startswith('.circuit')):
            flag = True
            continue
        elif(line.startswith('.end')):
            flag = False
        if(flag):
            relevant.append(line)  
    nodes = []
    values = {}
    for line in relevant:
        a = line.split()
        if(len(a)>1):
            for hm in range(2):
                if(a[hm+1] not in nodes):  #creating a list of all nodes
                    nodes.append(a[hm+1])
            if(a[0].startswith('R') | a[0].startswith('L') | a[0].startswith('C')):  #creating a dictionary where key is the element and 
                values[a[0]]=a[3]                                                    # value is the elements value/magnitude
            elif(a[0].startswith('V')): #an independent voltage source will also add a node
                if(a[3] == 'dc'):
                    values[a[0]]=a[4]
                    nodes.append(a[0])
                    dc+=1
                elif(a[3] == 'ac'):
                    values [a[0]] = str(int(a[4])*np.cos(float(a[5])))
                    nodes.append(a[0])
                    ac+=1
            elif(a[0].startswith('I')):
                if(a[3] == 'dc'):
                    values[a[0]]=a[4]
                    dc+=1
                elif(a[3] == 'ac'):
                    values [a[0]] = str(int(a[4])*np.cos(float(a[5])))
                    ac+=1
    n = len(nodes)-1
    if(dc*ac !=0):  #making sure that the circuit has either ac or dc sources, not both(multiple frequencies)
        return b
    frq = []
    freq_check = open(ckt)
    for line in freq_check:
        line = line.strip()
        line = line.split()
        if(len(line)>0):
            if(line[0] == '.ac'):
                if(line[2] not in frq):
                    frq.append(line[2])
    if(len(frq) > 1 ):  #checking for circuits with multiple frequencies
        return b
    if(ac!=0):
        freq = 2*math.pi*float(frq[0])
    circuit_matrix = []
    for i in range(n):
        circuit_matrix.append([0 for j in range(n)])
    const_matrix = [0 for i in range(n)]
    nodes.remove('GND')
    node_dict = {}
    t=0
    for i in nodes:
        node_dict[i] = str(t)
        t+=1 
    try:
        for node in nodes: #performing KCL at each node
            for line in relevant:
                line = line.split()
                if(node in line[1]):
                    if(line[0].startswith('R')): 
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += 1/(float(values[line[0]]))
                        if(line[2] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[2]])] -= 1/float(values[line[0]])
                    elif(line[0].startswith('I')):
                        const_matrix[int(node_dict[node])] += float(values[line[0]])
                    elif(line[0].startswith('V')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[line[0]])] = -1
                    elif(line[0].startswith('L')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += 1/complex(0,freq*float(values[line[0]]))
                        if(line[2] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[2]])] -= 1/complex(0,freq*float(values[line[0]]))
                    elif(line[0].startswith('C')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += complex(0,freq*float(values[line[0]]))
                        if(line[2] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[2]])] -= complex(0,freq*float(values[line[0]]))
                elif(node in line[2]):
                    if(line[0].startswith('R')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += 1/(float(values[line[0]]))
                        if(line[1] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[1]])] -= 1/float(values[line[0]])
                    elif(line[0].startswith('I')):
                        const_matrix[int(node_dict[node])] -= float(values[line[0]])
                    elif(line[0].startswith('V')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[line[0]])] = 1
                    elif(line[0].startswith('L')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += 1/complex(0,freq*float(values[line[0]]))
                        if(line[1] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[1]])] -= 1/complex(0,freq*float(values[line[0]]))
                    elif(line[0].startswith('C')):
                        circuit_matrix[int(node_dict[node])][int(node_dict[node])] += complex(0,freq*float(values[line[0]]))
                        if(line[1] != 'GND'):
                            circuit_matrix[int(node_dict[node])][int(node_dict[line[1]])] -= complex(0,freq*float(values[line[0]]))
                elif(node == line[0]):
                    if(line[1] != 'GND'):
                        circuit_matrix[int(node_dict[node])][int(node_dict[line[1]])] = 1
                    if(line[2] != 'GND'):
                        circuit_matrix[int(node_dict[node])][int(node_dict[line[2]])] = -1
                    const_matrix[int(node_dict[node])] = float(values[line[0]])
    except:
        print("Cannot analyse capacitors for DC inputs")

    eq_solve(circuit_matrix,const_matrix)

circuit_solve()  


Enter file containing circuit: ckt3.netlist


[-1.00000000e+01+0.j  4.97076023e-03+0.j -5.02923977e+00+0.j
 -2.57309942e+00+0.j -1.40350877e+00+0.j -9.35672515e-01+0.j]


The function circuit_solve() performs modified nodal analysis on a given .SPICE netlist and gives the voltage at each node, and the current flowing through the independent voltage sources.
<br>
We first find the number of nodes in the circuit and then perform KCL at each of the node. Then we update the coefficients of the variables to the circuit matrix and the constant matrix. Finally we solve the matrix and get the output.
<br>
>*Regarding the sign convention for nodal analysis, I have referred to the [following material](https://youtube.com/playlist?list=PLlsTHbFeR7v0QG8Q4l6oc7d---eKKSKTz)